# Summary
County census data is combined with 2016 Presidential election results. Data is cleaned for use with classification models. Election results are labeled as 0 or 1.

# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import glob

# Data

## Census Data
Census data was collected from statsamerica.org (http://www.statsamerica.org/USCP/) for the year 2017.

In [4]:
# import file with county code mapping
df_county_id = pd.read_csv('data/us_county_codes.csv')
df_county_id.head()

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


In [5]:
# remove trailing whitespace from column titles
df_county_id.columns = ['code', 'county', 'state']

In [6]:
df_county_id['state'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'AS', 'MP', 'GU', 'PR',
       'VI'], dtype=object)

In [7]:
# remove non-voting US territories
territories = ['AS', 'MP', 'GU', 'PR', 'VI']
df_county_id.drop(df_county_id[df_county_id['state'].isin(territories)].index, inplace=True)

In [8]:
local_path = 'data/statsamerica_data/'
cens_files = glob.glob(local_path + '*.csv')

In [9]:
# create new dataframe for merging census data
df_cens_data = df_county_id.copy()

# merge each file to dataframe
for file in cens_files:
    df_cens_data = pd.merge(df_cens_data, pd.read_csv(file).iloc[:, [1, 2]], 
                            on='code', how='left')

In [10]:
# import file with county population density
df_county_size = pd.read_csv('data/county_size.csv', encoding = "ISO-8859-1")
df_county_size.head()

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402
0,Id,Id2,Geography,Target Geo Id,Target Geo Id2,Geographic area,Geographic area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...
1,0100000US,NaN,United States,0100000US,NaN,United States,United States,308745538(r38234),131704730(r15031),3796742.23,264836.79,3531905.43,87.4,37.3
2,0100000US,NaN,United States,0400000US01,01,United States - Alabama,Alabama,4779736(r38235),2171853(r15032),52420.07,1774.74,50645.33,94.4,42.9
3,0100000US,NaN,United States,0500000US01001,01001,United States - Alabama - Autauga County,Autauga County,54571,22135,604.39,9.95,594.44,91.8,37.2
4,0100000US,NaN,United States,0500000US01003,01003,United States - Alabama - Baldwin County,Baldwin County,182265,104061,2027.31,437.53,1589.78,114.6,65.5


In [11]:
# select id and population density columns from dataframe
df_county_size = df_county_size.iloc[:, [4, -2]]
# rename columns
df_county_size.columns = ['code', 'pop_density']
# drop top three rows
df_county_size.drop(index=[0, 1, 2], inplace=True)
# set code as int data type
df_county_size['code'] = df_county_size['code'].astype('int')

In [12]:
# merge to dataframe
df_cens_data = pd.merge(df_cens_data, df_county_size, on='code', how='left')
df_cens_data.head()

,code,county,state,unempl_rate,hs_rate,median_age,hisp_pop,growth,per_cap_income,college_rate,hh_income,poverty_rate,labor_force,households,white_pop,population,pop_density
0,1001,Autauga,AL,3.9,87.60%,37.8,"1,416",1.70%,"$39,721",24.60%,"$54,487",13.5,"25,909","20,800","42,311","55,504",91.8
1,1003,Baldwin,AL,4.0,90.00%,42.3,"8,712",16.70%,"$41,286",29.50%,"$56,460",11.7,"91,567","75,149","172,441","212,628",114.6
2,1005,Barbour,AL,5.9,73.80%,38.7,"1,147",-8.00%,"$31,788",12.90%,"$32,884",29.9,"8,236","9,122","12,430","25,270",31.0
3,1007,Bibb,AL,4.4,80.70%,40.2,502,-1.10%,"$29,264",12.00%,"$43,079",20.1,"8,506","7,048","17,370","22,668",36.8
4,1009,Blount,AL,4.0,80.00%,40.8,"5,036",1.20%,"$31,470",13.00%,"$47,213",14.1,"24,494","20,619","55,073","58,013",88.9


## Election Data
2016 presidential election results at the county level were collected from Tony McGovern's GitHub page (https://github.com/tonmcg/County_Level_Election_Results_12-16). These results provide the difference as a percentage between the Democratic and Replican candidates. This value is to be used as the target variable for classification.

In [13]:
# import 2016 election results by county
df_election = pd.read_csv('data/2016_US_County_Level_Presidential_Results.csv', index_col=0)
df_election.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [14]:
# reduce to county code and election percentage difference
df_election = df_election.iloc[:, [6, -1]]
df_election.columns = ['percent_diff', 'code']
df_election.head()

,percent_diff,code
0,15.17%,2013
1,15.17%,2016
2,15.17%,2020
3,15.17%,2050
4,15.17%,2060


# Process Data

## Merge census and election data

In [15]:
df_counties = pd.merge(df_cens_data, df_election, on='code', how='left')
df_counties.head()

,code,county,state,unempl_rate,hs_rate,median_age,hisp_pop,growth,per_cap_income,college_rate,hh_income,poverty_rate,labor_force,households,white_pop,population,pop_density,percent_diff
0,1001,Autauga,AL,3.9,87.60%,37.8,"1,416",1.70%,"$39,721",24.60%,"$54,487",13.5,"25,909","20,800","42,311","55,504",91.8,49.48%
1,1003,Baldwin,AL,4.0,90.00%,42.3,"8,712",16.70%,"$41,286",29.50%,"$56,460",11.7,"91,567","75,149","172,441","212,628",114.6,57.79%
2,1005,Barbour,AL,5.9,73.80%,38.7,"1,147",-8.00%,"$31,788",12.90%,"$32,884",29.9,"8,236","9,122","12,430","25,270",31.0,5.61%
3,1007,Bibb,AL,4.4,80.70%,40.2,502,-1.10%,"$29,264",12.00%,"$43,079",20.1,"8,506","7,048","17,370","22,668",36.8,55.54%
4,1009,Blount,AL,4.0,80.00%,40.8,"5,036",1.20%,"$31,470",13.00%,"$47,213",14.1,"24,494","20,619","55,073","58,013",88.9,81.38%


## Formatting

In [16]:
# function to reformat monetary and count values
def clean_num(num_str):
    try:
        return(int(str(num_str).replace('$', '').replace(',', '')))
    except:
        return(np.nan)

In [17]:
# function to reformat rate values
def clean_rate(rate_str):
    try:
        return(float(str(rate_str).replace('%', '').strip()))
    except:
        return(np.nan)

In [18]:
# monetary and count columns
num_cols = df_counties.columns[[6, 8, 10, 12, 13, 14, 15]]
df_counties[num_cols] = df_counties[num_cols].applymap(clean_num)
# rate columns
rate_cols = df_counties.columns[[4, 7, 9, 17]]
df_counties[rate_cols] = df_counties[rate_cols].applymap(clean_rate)

In [19]:
# check data types per column
df_counties.dtypes

code                int64
county             object
state              object
unempl_rate       float64
hs_rate           float64
median_age        float64
hisp_pop          float64
growth            float64
per_cap_income    float64
college_rate      float64
hh_income         float64
poverty_rate      float64
labor_force       float64
households        float64
white_pop         float64
population        float64
pop_density        object
percent_diff      float64
dtype: object

In [20]:
# convert numeric columns from object to float64
df_counties['pop_density'] = df_counties['pop_density'].astype('float')

## White/Hispanic Rates
Replace white and hispanic population with proportion

In [21]:
# white rate for county
df_counties['white_rate'] = df_counties['white_pop'] / df_counties['population']
# drop white population
df_counties.drop('white_pop', axis=1, inplace=True)
# hispanic rate for county
df_counties['hisp_rate'] = df_counties['hisp_pop'] / df_counties['population']
# drop white population
df_counties.drop('hisp_pop', axis=1, inplace=True)

In [22]:
df_counties.head()

,code,county,state,unempl_rate,hs_rate,median_age,growth,per_cap_income,college_rate,hh_income,poverty_rate,labor_force,households,population,pop_density,percent_diff,white_rate,hisp_rate
0,1001,Autauga,AL,3.9,87.6,37.8,1.7,39721.0,24.6,54487.0,13.5,25909.0,20800.0,55504.0,91.8,49.48,0.762305,0.025512
1,1003,Baldwin,AL,4.0,90.0,42.3,16.7,41286.0,29.5,56460.0,11.7,91567.0,75149.0,212628.0,114.6,57.79,0.810999,0.040973
2,1005,Barbour,AL,5.9,73.8,38.7,-8.0,31788.0,12.9,32884.0,29.9,8236.0,9122.0,25270.0,31.0,5.61,0.491888,0.045390
3,1007,Bibb,AL,4.4,80.7,40.2,-1.1,29264.0,12.0,43079.0,20.1,8506.0,7048.0,22668.0,36.8,55.54,0.766278,0.022146
4,1009,Blount,AL,4.0,80.0,40.8,1.2,31470.0,13.0,47213.0,14.1,24494.0,20619.0,58013.0,88.9,81.38,0.949322,0.086808


## Class Labels

In [23]:
# classify swing county as '1' and safe county as '0'
thresh = 5 # define close election threshold
df_counties['close_elect'] = pd.DataFrame(np.where(df_counties['percent_diff'] <= thresh, 1, 0))
df_counties.drop('percent_diff', axis=1, inplace=True)

In [24]:
df_counties.head()

,code,county,state,unempl_rate,hs_rate,median_age,growth,per_cap_income,college_rate,hh_income,poverty_rate,labor_force,households,population,pop_density,white_rate,hisp_rate,close_elect
0,1001,Autauga,AL,3.9,87.6,37.8,1.7,39721.0,24.6,54487.0,13.5,25909.0,20800.0,55504.0,91.8,0.762305,0.025512,0
1,1003,Baldwin,AL,4.0,90.0,42.3,16.7,41286.0,29.5,56460.0,11.7,91567.0,75149.0,212628.0,114.6,0.810999,0.040973,0
2,1005,Barbour,AL,5.9,73.8,38.7,-8.0,31788.0,12.9,32884.0,29.9,8236.0,9122.0,25270.0,31.0,0.491888,0.045390,0
3,1007,Bibb,AL,4.4,80.7,40.2,-1.1,29264.0,12.0,43079.0,20.1,8506.0,7048.0,22668.0,36.8,0.766278,0.022146,0
4,1009,Blount,AL,4.0,80.0,40.8,1.2,31470.0,13.0,47213.0,14.1,24494.0,20619.0,58013.0,88.9,0.949322,0.086808,0


## NaN values

In [25]:
len(df_counties)

3142

In [26]:
# nan values per column
df_counties.isna().sum()

code               0
county             0
state              0
unempl_rate       10
hs_rate           10
median_age        10
growth            61
per_cap_income    37
college_rate      10
hh_income         10
poverty_rate      10
labor_force       10
households        10
population        61
pop_density        7
white_rate        63
hisp_rate         63
close_elect        0
dtype: int64

In [27]:
# impute mean for nan values
df_counties.fillna(df_counties.mean().astype(float), inplace=True)

## Check overview of data

In [28]:
df_counties.describe()

,code,unempl_rate,hs_rate,median_age,growth,per_cap_income,college_rate,hh_income,poverty_rate,labor_force,households,population,pop_density,white_rate,hisp_rate,close_elect
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.00000,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3142.000000,3142.000000
mean,30433.846913,4.603544,85.804087,41.023914,0.769847,40722.920773,20.785089,49510.41060,15.877203,5.081511e+04,3.730124e+04,1.036017e+05,259.021308,0.834376,0.089015,0.051241
std,15152.105476,1.630022,6.530346,5.326632,6.711405,11261.696574,9.113593,12854.92018,6.253065,1.659372e+05,1.128825e+05,3.289332e+05,1724.116500,0.167222,0.134882,0.220525
min,1001.000000,1.600000,48.500000,21.500000,-33.500000,0.000000,3.000000,22045.00000,3.400000,1.000000e+02,3.700000e+01,1.340000e+02,0.000000,0.090114,0.000000,0.000000
25%,19001.500000,3.500000,81.700000,37.800000,-3.300000,33973.750000,14.400000,41078.00000,11.400000,4.974500e+03,4.250500e+03,1.112650e+04,17.000000,0.765350,0.019783,0.000000
50%,29184.000000,4.400000,87.200000,41.023914,-0.400000,38754.500000,18.600000,47596.00000,14.900000,1.167750e+04,9.901000e+03,2.666800e+04,45.350000,0.886850,0.039253,0.000000
75%,45086.500000,5.300000,90.700000,44.100000,3.600000,44779.500000,24.600000,55262.25000,19.100000,3.182500e+04,2.615625e+04,7.501275e+04,114.925000,0.954885,0.089015,0.000000
max,56045.000000,19.100000,98.700000,66.000000,100.100000,199635.000000,80.200000,134609.00000,48.600000,5.123933e+06,3.281845e+06,1.016351e+07,69468.400000,1.428782,0.967388,1.000000


# Reindex

In [29]:
# set index to county code
df_counties.set_index('code', inplace=True)

# Export to CSV

In [29]:
df_counties.to_csv('data/county_data.csv')